In [1]:
%use dataframe, kandy

In [2]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
import java.nio.file.Path
import kotlin.io.path.*

fun String.parse(s1: String, s2: String): Pair<Int, Int> {
    return Regex("""$s1 = (\d+), $s2 = (\d+)""").find(this)!!.groupValues.let {
        it[1].toInt() to it[2].toInt()
    }
}

fun String.parseTime(s1: String, s2: String): Pair<Long, Long> {
    return Regex("""$s1 = (\d+) ms, $s2 = (\d+) ms""").find(this)!!.groupValues.let {
        it[1].toLong() to it[2].toLong()
    }
}

fun mkDataFrameFromLog(path: Path): DataFrame<*> {
    val benchmark = mutableListOf<String>()
    val cnt = mutableListOf<Int>()
    val sat = mutableListOf<Int>()
    val unsat = mutableListOf<Int>()
    val unknown = mutableListOf<Int>()
    val reusedUnsat = mutableListOf<Int>()
    val solvingTime = mutableListOf<Long>()
    val checkingTime = mutableListOf<Long>()
    val updatingTime = mutableListOf<Long>()

    path.readLines()
        .chunked(8)
        .dropLastWhile { it.size != 8 }
        .forEach {
            val benchmarkR = it[1].substringBefore(" ")
            val (cntR, satR) = it[2].parse("cnt", "sat")
            val (unsatR, unknownR) = it[3].parse("unsat", "unknown")
            val reusedUnsatR = Regex("reusedUnsat = (\\d+)").find(it[4])!!.groupValues[1].toInt()
            val (solvingTimeR, checkingTimeR) = it[5].parseTime("solvingTime", "checkingTime")
            val (updatingTimeR, _) = it[6].parseTime("updatingTime", "totalTime")

            benchmark += benchmarkR
            cnt += cntR
            sat += satR
            unsat += unsatR
            unknown += unknownR
            reusedUnsat += reusedUnsatR
            solvingTime += solvingTimeR
            checkingTime += checkingTimeR
            updatingTime += updatingTimeR
        }
    return dataFrameOf(
        "benchmark" to benchmark,
        "cnt" to cnt,
        "sat" to sat,
        "unsat" to unsat,
        "unknown" to unknown,
        "reusedUnsat" to reusedUnsat,
        "solvingTime" to solvingTime,
        "checkingTime" to checkingTime,
        "updatingTime" to updatingTime,
    )
}

In [4]:
fun DataFrame<*>.addConfigColumn(name: String): DataFrame<*> =
    add(column(List(rowsCount()) { name }).rename("config"))

fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

fun DataFrame<*>.timeToLong(): DataFrame<*> = columns().filter { it.name.endsWith("Time") }.toTypedArray().let { cols ->
    remove(*cols).addAll(cols.map { it.convertToLong() })
}

val folder = "../output/cache"

val data = Path(folder).listDirectoryEntries().map {
    if (it.div("smtData.csv").exists()) {
        DataFrame.readCSV((it / "smtData.csv").toFile()).timeToLong()
    } else {
        mkDataFrameFromLog(it / "cache-a-lot.log")
    }.addConfigColumn(it.name).addProjectColumn().sortBy("project")
}.concat()

data.columnNames()

[benchmark, cnt, sat, unsat, unknown, reusedUnsat, solvingTime, checkingTime, updatingTime, config, project]

In [5]:
val configs = data.config.countDistinct()
val fullBenchs = data.groupBy("benchmark")
    .aggregate { it.config.countDistinct() into "configs" }
    .filter { it["configs"] == configs }
    .benchmark.values.toList()
val fullData = data.filter { benchmark in fullBenchs }

In [6]:
fullData
    .filter { !config.endsWith("tt") && config != "empty" }
    .groupBy("config", "project")
    .aggregate {
        val total = unsat.sum()
        val reused = reusedUnsat.sum()
        (total - reused).toDouble() / total into "unique unsat"
        reused.toDouble() / total into "reused unsat"
    }
    .gather("reused unsat", "unique unsat")
    .into("name", "value")
    .sortBy("project", "config")
    .groupBy("name")
    .plot {
        val name = "FASTJSON"
        val indices = project.indices.filter { project[it] == name }
        bars {
            x(config.get(indices))
            y(get("value").get(indices))
            fillColor(get("name").get(indices)) {
                scale = categorical(
                    "unique unsat" to Color.GREY,
                    "reused unsat" to Color.ORANGE,
                )
            }
            position = Position.stack()
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="S72QiU"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#a39999","#fc8452"],
"limits":["unique unsat","reused unsat"]
}],
"layers":[{
"mapping":{
"x":"config",
"y":"value",
"fill":"name"
},
"stat":"identity",
"data":{
"name":["reused unsat","reused unsat","reused unsat","unique unsat","unique unsat","unique unsat"],
"value":[0.3153153153153153,0.33153153153153153,0.23963963963963963,0.6846846846846847,0.6684684684684684,0.7603603603603604],
"config":["full-random","simple-full","simple-random","full-random","simple-full","simple-random"]
},
"sampling":"none",
"position":"stack",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("S72QiU");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 full-random 
 
 
 
 
 
 
 
 
 simple-full 
 
 
 
 
 
 
 
 
 simple-random 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 0.6 
 
 
 
 
 
 
 0.8 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 value 
 
 
 
 
 config 
 
 
 
 
 
 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 unique unsat 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 reused unsat

In [7]:
fullData
    .filter { config == "full-random" }
    .groupBy("project", "config")
    .aggregate {
        (solvingTime.sum().toDouble() / cnt.sum()) into "solvingTime"
        (checkingTime.sum().toDouble() / cnt.sum()) into "checkingTime"
        (updatingTime.sum().toDouble() / cnt.sum()) into "updatingTime"
    }
    .gather(
        "solvingTime", 
        "checkingTime", 
        "updatingTime",
    )
    .into("type", "value")
    .sortBy("project", "config")
    .groupBy("type")
    .plot {
        bars {
            x(project.values.zip(config.values))
            y("value") {
                axis.name = "avg. time (ms)"
            }
            fillColor("type") {
                scale = categorical(
                    "solvingTime" to Color.ORANGE,
                    "checkingTime" to Color.GREY,
                    "updatingTime" to Color.BLUE,
                )
            }
            position = Position.stack()
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="IAm6UG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"x":[["FASTJSON","full-random"],["GUAVA","full-random"],["SEATA","full-random"],["SPOON","full-random"],["FASTJSON","full-random"],["GUAVA","full-random"],["SEATA","full-random"],["SPOON","full-random"],["FASTJSON","full-random"],["GUAVA","full-random"],["SEATA","full-random"],["SPOON","full-random"]],
"type":["solvingTime","solvingTime","solvingTime","solvingTime","checkingTime","checkingTime","checkingTime","checkingTime","updatingTime","updatingTime","updatingTime","updatingTime"],
"value":[470.36400986031225,431.4437340153453,95.87610619469027,862.435736677116,37221.8742810189,64581.36061381074,3205.858407079646,100935.94670846395,5.599013968775678,2.846547314578005,1.7168141592920354,3.156739811912226]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. time (ms)",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#fc8452","#a39999","#5470c6"],
"limits":["solvingTime","checkingTime","updatingTime"]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"value",
"fill":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("IAm6UG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [FASTJSON, full-random] 
 
 
 
 
 
 
 
 
 [GUAVA, full-random] 
 
 
 
 
 
 
 
 
 [SEATA, full-random] 
 
 
 
 
 
 
 
 
 [SPOON, full-random] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 avg. time (ms) 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 solvingTime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 checkingTime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 updatingTime

In [8]:
fullData
    .filter { !config.endsWith("tt") }
    .sortBy("project", "config")
    .plot {
        boxplot(
            column<String>(
                project.values.zip(config.values).map { (p, c) -> "$p $c" }
            ).named("project config"),
            column<Double>(
                solvingTime.values.zip(cnt.values).map { it.first.toDouble() / it.second }
            ).named("avg. solving time (ms)")
        )
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="5NU3ZJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"project config",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"avg. solving time (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"y":"y"
},
"stat":"identity",
"data":{
"middle":[388.10287822878234,286.51658536585364,368.92693726937273,390.4943173431734,262.275,262.95,265.35,241.825,27.38157894736842,17.605263157894736,22.486842105263158,27.789473684210527,1106.8214285714287,990.8214285714286,1044.107142857143,1066.5],
"min":[7.2823529411764705,1.4336283185840708,1.2212389380530972,1.584070796460177,131.76076555023923,132.91387559808612,124.84210526315789,133.244019138756,8.0,4.0,6.0,5.0,272.0,256.4166666666667,269.6666666666667,212.16666666666666],
"max":[1052.6503496503497,875.7202797202797,930.9370629370629,1013.3706293706293,433.6521739130435,396.71014492753625,431.768115942029,452.0217391304348,292.3888888888889,263.6666666666667,267.8611111111111,333.8333333333333,2465.6,2391.2,2376.8,2449.866666666667],
"lower":[12.78125,9.713970588235295,7.297841051314142,10.740644555694619,181.8421052631579,181.56140350877192,184.0,181.42105263157896,8.0,4.0,6.0,5.0,308.62595419847327,306.48854961832063,297.80152671755724,306.2519083969466],
"upper":[735.774226006192,627.9481679290477,653.7293085655315,713.4294633642932,433.6521739130435,396.71014492753625,431.768115942029,452.0217391304348,292.3888888888889,263.6666666666667,267.8611111111111,333.8333333333333,1845.7391304347825,1516.608695652174,1613.304347826087,1765.9565217391305],
"x":["FASTJSON empty","FASTJSON full-random","FASTJSON simple-full","FASTJSON simple-random","GUAVA empty","GUAVA full-random","GUAVA simple-full","GUAVA simple-random","SEATA empty","SEATA full-random","SEATA simple-full","SEATA simple-random","SPOON empty","SPOON full-random","SPOON simple-full","SPOON simple-random"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["GUAVA empty","GUAVA full-random","GUAVA simple-full","GUAVA simple-random"],
"y":[1293.076923076923,1155.8284023668639,1123.3905325443786,1306.2781065088757]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("5NU3ZJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 FASTJSON empty 
 
 
 
 
 
 
 
 
 FASTJSON full-random 
 
 
 
 
 
 
 
 
 FASTJSON simple-full 
 
 
 
 
 
 
 
 
 FASTJSON simple-random 
 
 
 
 
 
 
 
 
 GUAVA empty 
 
 
 
 
 
 
 
 
 GUAVA full-random 
 
 
 
 
 
 
 
 
 GUAVA simple-full 
 
 
 
 
 
 
 
 
 GUAVA simple-random 
 
 
 
 
 
 
 
 
 SEATA empty 
 
 
 
 
 
 
 
 
 SEATA full-random 
 
 
 
 
 
 
 
 
 SEATA simple-full 
 
 
 
 
 
 
 
 
 SEATA simple-random 
 
 
 
 
 
 
 
 
 SPOON empty 
 
 
 
 
 
 
 
 
 SPOON full-random 
 
 
 
 
 
 
 
 
 SPOON simple-full 
 
 
 
 
 
 
 
 
 SPOON simple-random 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [9]:
fullData.groupBy("project", "config")
    .aggregate { 
        solvingTime.sum().toDouble() / cnt.sum() into "solvingTime"
        checkingTime.sum().toDouble() / cnt.sum() into "checkingTime"
    }.sortBy("project", "config").filter { !config.endsWith("tt") }

project,config,solvingTime,checkingTime
FASTJSON,empty,"538,040263","0,000000"
FASTJSON,full-random,"470,364010","37221,874281"
FASTJSON,simple-full,"488,373870","1,632703"
FASTJSON,simple-random,"522,843878","0,350041"
GUAVA,empty,"467,236573","0,000000"
GUAVA,full-random,"431,443734","64581,360614"
GUAVA,simple-full,"428,585678","0,896419"
GUAVA,simple-random,"470,991049","0,287724"
SEATA,empty,"111,637168","0,000000"
SEATA,full-random,"95,876106","3205,858407"


In [19]:
fullData
    .filter { project == "FASTJSON" }
    .sortBy("benchmark")
    .plot {
        val i1 = config.indices.filter { config[it] == "full-random" }
        val i2 = config.indices.filter { config[it] == "simple-full" }
        points {
            x(checkingTime.get(i2).values.zip(cnt.get(i2).values).map { it.first.toDouble() / it.second })
            y(checkingTime.get(i1).values.zip(cnt.get(i1).values).map { it.first.toDouble() / it.second })
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3n7lfX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.049019607843137254,1.75,0.055944055944055944,6.12280701754386,0.3902439024390244,0.43911439114391143,0.06382978723404255,0.5663716814159292,0.0,0.13333333333333333],
"y":[16070.392156862745,1.625,15265.405594405594,106379.72982456141,112.07317073170732,39198.10701107011,1618.6382978723404,24.07079646017699,26.68235294117647,4989.733333333334]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("3n7lfX");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

In [32]:
import org.jetbrains.kotlinx.statistics.stats.mean

val t = data.filter { config == "simple-full" }.let { it.filter { benchmark in fullBenchs }.checkingTime.sum().toDouble() / it.filter { benchmark !in fullBenchs }.checkingTime.sum() }
t

0.019794794438673137